## Imports

In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession

## Start Spark Session

In [2]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Leitura de Arquivo de Formato Fixo") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Variables

In [3]:
# Define caminho local onde será armazenada a tabela Delta
delta_path_cotahist = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/cotahist"
delta_path_cotacao_historica = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver/cotacao_historica"    

## Create table bronze

In [4]:
## Create table cotahist
spark.sql(f"""
CREATE TABLE IF NOT EXISTS cotahist (
  tipo_registro STRING,
  data_pregao STRING,
  codigo_bdi STRING,
  codigo_negociacao STRING,
  tipo_mercado STRING,
  nome_resumido_empresa STRING,
  especificacao_papel STRING,
  prazo_dias_mercado STRING,
  moeda_referencia STRING,
  preco_abertura_papel STRING,
  preco_maximo_papel STRING,
  preco_minimo_papel STRING,
  preco_medio_papel STRING,
  preco_ultimo_negocio STRING,
  preco_melhor_oferta_compra STRING,
  preco_melhor_oferta_venda STRING,
  numero_negocios_efetuados STRING,
  quantidade_total_titulos STRING,
  volume_total_titulos STRING,
  preco_exercicio_opcoes STRING,
  indicador_correcao_precos STRING,
  data_vencimento_opcoes STRING,
  fator_cotacao_papel STRING,
  preco_exercicio_pontos STRING,
  codigo_papel_sistema STRING,
  numero_distribuicao_papel STRING,
  data_insercao TIMESTAMP
)
USING DELTA
LOCATION '{delta_path_cotahist}'
""")

DataFrame[]

## Create table silver

In [5]:
## Create table cotacao_historica
spark.sql(f"""
CREATE TABLE IF NOT EXISTS cotacao_historica (
  tipo_registro STRING,
  data_pregao DATE,
  codigo_bdi STRING,
  codigo_negociacao STRING,
  tipo_mercado STRING,
  nome_resumido_empresa STRING,
  especificacao_papel STRING,
  prazo_dias_mercado STRING,
  moeda_referencia STRING,
  preco_abertura_papel DECIMAL(11,2),
  preco_maximo_papel DECIMAL(11,2),
  preco_minimo_papel DECIMAL(11,2),
  preco_medio_papel DECIMAL(11,2),
  preco_ultimo_negocio DECIMAL(11,2),
  preco_melhor_oferta_compra DECIMAL(11,2),
  preco_melhor_oferta_venda DECIMAL(11,2),
  numero_negocios_efetuados INT,
  quantidade_total_titulos INT,
  volume_total_titulos DECIMAL(16,2),
  preco_exercicio_opcoes DECIMAL(11,2),
  indicador_correcao_precos STRING,
  data_vencimento_opcoes DATE,
  fator_cotacao_papel STRING,
  preco_exercicio_pontos DECIMAL(7,6),
  codigo_papel_sistema STRING,
  numero_distribuicao_papel STRING,
  data_insercao TIMESTAMP
)
USING DELTA
LOCATION '{delta_path_cotacao_historica}'
""")

DataFrame[]

## Stop Spark Session

In [6]:
# Encerra a SparkSession
spark.stop()